# Introducción
En este notebook, exploramos diferentes modelos de procesamiento de lenguaje natural (NLP) para construir un recomendador de canciones basado en letras y estados de ánimo. Evaluamos varios modelos y finalmente elegimos el que mejor se adapta a nuestras necesidades.

**Quisiera pedir perdón ya que es un notebook recuperado que ha perdido su mayor esencia y mucho del trabajo que llevaba (gajes de ser nueva en el mundo git)**

En el cuaderno [modelos_perdido.ipynb](functions/4%20-%20Recomendador%20letras%20canciones/modelos_perdido.ipynb) corrompido, salen algunos de los resultados de los modelos, aunque las métricas de algunos eran mayores en similaridad el que más nos ha convencido finalmente ha sido el de Roberta el cual en cuaderno [Berta_Final.ipynb](functions/4%20-%20Recomendador%20letras%20canciones/Berta_Final.ipynb) podréis ver que lo hemos mezclado con etiqutas FAISS para una búsqueda más rápida.


# 📌 Comparación de Modelos para Búsqueda Semántica de Canciones

| **Modelo**                  | **Ventajas**                                         | **Desventajas**                                       | **Recomendado?** | **Dimensión Embeddings** |
|-----------------------------|----------------------------------------------------|-----------------------------------------------------|-------------------------|--------------------------|
| **✅ all-roberta-large-v1**  | 🔹 Muy preciso en textos largos. 🔹 Captura contexto profundo. | 🔹 Más lento en inferencia. 🔹 Requiere más RAM/GPU. | ✅ **Sí, es el modelo que hemos elegido por su alta precisión.** | 1024 |
| **✅ sentence-t5-base**      | 🔹 Optimizado para búsqueda semántica. 🔹 Puede generar textos (opcional). | 🔹 No está entrenado en datos musicales. | ✅ **Sí, es nuestra segunda opción por su equilibrio entre precisión y eficiencia.** | 768 |
| **🔥 all-mpnet-base-v2**     | 🔹 Muy equilibrado: rápido y preciso. 🔹 Optimizado para comparación semántica. | 🔹 No es el más avanzado, pero tiene buen rendimiento. | ⚠️ **Tal vez, si queremos una opción rápida y precisa.** | 768 |
| **🔥 all-mpnet-large**       | 🔹 Mismo modelo que usamos pero más preciso. 🔹 Optimizado para comprensión profunda. | 🔹 Puede ser más lento que `mpnet-base-v2`. | ⚠️ **Roberta Nos gusta más para nuestra finalidad.** | 1024 |
| **🔥 all-distilroberta-v1**  | 🔹 Más rápido que `roberta-large`. 🔹 Muy bueno en búsqueda semántica. | 🔹 No es tan profundo como `roberta-large`. | ⚠️ **Tal vez, si necesitamos más velocidad sin perder mucha precisión.** | 768 |
| **Bert**                    | 🔹 Bien entrenado en datos generales. 🔹 Buen rendimiento en tareas generales de NLP. | 🔹 No tan optimizado para datos musicales específicos. | ❌ **No, preferimos modelos más especializados.** | 768 |


## Intento de Red Neuronal para Juntar Embeddings de T5, RoBERTa y MPNet
Se intentó utilizar una red neuronal para combinar los embeddings generados por los modelos T5, RoBERTa y MPNet con el objetivo de mejorar la precisión de las recomendaciones. Sin embargo, este enfoque no resultó ser eficiente en términos de tiempo de procesamiento y rendimiento. Por lo tanto, se decidió no utilizar esta técnica en la implementación final.

## Descarte del Clustering y Formateo Total del Texto

### Clustering
Inicialmente, consideramos utilizar técnicas de clustering para agrupar canciones con letras similares. Sin embargo, este enfoque fue descartado por varias razones:

1. **Complejidad Computacional:** El clustering de grandes volúmenes de datos textuales requiere una cantidad significativa de recursos computacionales, lo que puede ser ineficiente y lento.
2. **Pérdida de Información Semántica:** Los métodos de clustering tradicionales pueden no capturar adecuadamente la riqueza semántica y el contexto de las letras de las canciones, lo que resulta en agrupaciones menos precisas.
3. **Dificultad en la Interpretación:** Los resultados del clustering pueden ser difíciles de interpretar y explicar, especialmente cuando se trata de datos textuales complejos como las letras de canciones.

### Formateo Total del Texto
También se probó a realizar un formateo exhaustivo de las letras de las canciones para normalizar el texto. Sin embargo, este enfoque fue descartado por las siguientes razones:

1. **Pérdida de Contexto:** El formateo excesivo puede eliminar información contextual importante, como la estructura poética y las expresiones idiomáticas, que son cruciales para la comprensión semántica.
2. **Complejidad del Preprocesamiento:** Implementar un formateo total del texto requiere un preprocesamiento complejo y detallado, lo que puede ser propenso a errores y difícil de mantener.
3. **Impacto Negativo en el Rendimiento del Modelo:** La normalización excesiva del texto puede afectar negativamente el rendimiento de los modelos de lenguaje, que están diseñados para manejar variaciones en el texto natural.

En lugar de estos enfoques, optamos por utilizar modelos de lenguaje avanzados como **all-roberta-large-v1** y **sentence-t5-base**, que pueden capturar de manera más efectiva la semántica y el contexto de las letras de las canciones sin necesidad de un preprocesamiento excesivo.

## Importar Librerías
Importamos las librerías necesarias para el análisis y procesamiento de datos.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import re
import pandas as pd
from tqdm import tqdm
from sentence_transformers import SentenceTransformer
import os

In [ ]:
import torch
print(torch.cuda.is_available())  # Debe devolver True si hay GPU disponible

## Cargar el Dataset
Cargamos el archivo CSV que contiene las letras de las canciones y otros metadatos.

In [ ]:
# Cargar el archivo CSV
file_path = r'C:\Users\solan\Downloads\get_data_from_songs\src\final_df.csv'
df = pd.read_csv(file_path)

## Información del Dataset
Mostramos información general del dataset, incluyendo el número de entradas y las columnas disponibles.

In [ ]:
df.info()

## Limpieza de Datos
Realizamos una limpieza básica de los datos, eliminando duplicados y manejando valores nulos.

In [ ]:
# Eliminar duplicados
df.drop_duplicates(inplace=True)

# Manejar valores nulos
df.fillna('', inplace=True)

## Preprocesamiento de Texto
Definimos funciones para limpiar y formatear las letras de las canciones.

In [ ]:
def clean_text(text):
    if pd.isna(text):
        return ""
    text = text.lower()  # Convertir a minúsculas
    text = re.sub(r'\s+', ' ', text)  # Reemplazar múltiples espacios con uno solo
    text = re.sub(r'[^a-zA-Z0-9 ]', '', text)  # Eliminar caracteres especiales
    text = text.strip()  # Eliminar espacios al inicio y final
    return text

def format_text_columns(df, columns):
    for col in columns:
        df[col] = df[col].astype(str).apply(clean_text)
    return df

In [ ]:
columns_to_format = ['song_name', 'artist_name', 'album_name', 'playlists_names', 'combined_genres', 'processed_lyrics']
df = format_text_columns(df, columns_to_format)

## Exploración de Datos
Realizamos una exploración inicial de los datos para entender mejor su estructura y contenido.

          artist_name                song_name  \
91   birds of chicago         estrella goodbye   
109         black rob              muscle game   
134      balsam range          trains i missed   
135            lights     cactus in the valley   
167    ray lamontagne         change your mind   
181     steve winwood           theres a river   
190      the vaccines           young american   
280         alpha 520     les larmes du soleil   
300             loote   high without your love   
375        tom t hall  legend of the lady bear   

                                               spotify_url  \
91   https://open.spotify.com/track/5jcetnvnAWGTMGufJ7oEnc   
109  https://open.spotify.com/track/5xYb779c1wmOmbfaexX8JK   
134  https://open.spotify.com/track/1ap81IwnNa5QeuvMMnbZfu   
135  https://open.spotify.com/track/288xqEJiCcSc2zGFanlclV   
167  https://open.spotify.com/track/66r7ecZd8LGtJWb9t6PhA6   
181  https://open.spotify.com/track/6IbNMlVEr2GY2DpkxINOts   
190  https://open.spotify.com/track/4gcZVTYjOBMh1UVlZy2YJW   
280  https://open.spotify.com/track/3ItQEhTWSvMOHTinqdxGB0   
300  https://open.spotify.com/track/3hwyRkhcVkcgUJHKKmxS8k   
375  https://open.spotify.com/track/4jWcTUjobvn6pzgN3b7w7l  

## Información del Dataset
Mostramos información general del dataset, incluyendo el número de entradas y las columnas disponibles.

In [ ]:
def clean_text(text):
    if pd.isna(text):
        return ""
    text = text.lower()  # Convertir a minúsculas
    text = re.sub(r'\s+', ' ', text)  # Reemplazar múltiples espacios con uno solo
    text = re.sub(r'[^a-zA-Z0-9 ]', '', text)  # Eliminar caracteres especiales
    text = text.strip()  # Eliminar espacios al inicio y final
    return text

def format_text_columns(df, columns):
    for col in columns:
        df[col] = df[col].astype(str).apply(clean_text)
    return df

In [ ]:
columns_to_format = ['song_name', 'artist_name', 'album_name', 'playlists_names', 'combined_genres', 'processed_lyrics']
df = format_text_columns(df, columns_to_format)

In [ ]:
# Nulos
pd.set_option('display.max_rows', None) 
df.isnull().sum()

In [ ]:
pd.set_option('display.max_rows', 30) 
# visualizar contenido de la celda spotify_url entero cuando se filtra en pandas
pd.set_option('display.max_colwidth', None)


In [ ]:
# Filtrar canciones con "mood_sad" alto y palabras clave en las letras
keywords = ["playa", "mar", "océano", "olas", "arena", "brisa", "agua"]
filtered_songs = df[(df['mood_sad'] > 0.7) & df['processed_lyrics'].str.contains('|'.join(keywords), case=False, na=False)]

# Mostrar las primeras 10 filas
print(filtered_songs[['artist_name', 'song_name', 'spotify_url', 'processed_lyrics']].head(10))


## PRUEBAS DE MODELOS

# all-MiniLM-L6-v2 - 384 Emb

**Descripción del Modelo:**
El modelo **all-MiniLM-L6-v2** es una versión ligera y eficiente de los modelos de lenguaje. Está diseñado para generar embeddings de alta calidad con una dimensión de 384. Este modelo es conocido por su rapidez y eficiencia en el uso de recursos, lo que lo hace ideal para aplicaciones en tiempo real y sistemas con limitaciones de memoria y procesamiento.

**Ventajas:**
- **Rápido y Ligero:** Debido a su menor tamaño, el modelo es extremadamente rápido en la generación de embeddings y consume menos memoria RAM y GPU.
- **Eficiente:** Ideal para aplicaciones que requieren respuestas rápidas y en tiempo real, como chatbots y sistemas de búsqueda rápida.

**Desventajas:**
- **Menor Precisión:** Aunque es rápido y eficiente, la precisión de los embeddings generados por **all-MiniLM-L6-v2** es menor en comparación con modelos más grandes y complejos.
- **Menor Capacidad de Comprensión:** No captura el contexto y la semántica de los textos tan bien como otros modelos más avanzados.

**Razón para No Usarlo en el Proyecto:**
Para este proyecto, que implica la recomendación de canciones basadas en letras y estados de ánimo o frases, se requiere un modelo que pueda capturar de manera precisa y profunda el contexto y la semántica de las letras de las canciones. Aunque **all-MiniLM-L6-v2** es rápido y eficiente, su menor precisión y capacidad de comprensión no son suficientes para las necesidades del proyecto. Modelos como **all-roberta-large-v1** y **sentence-t5-base** ofrecen una mejor comprensión semántica y precisión, lo que los hace más adecuados para este tipo de tareas.

In [ ]:
from langdetect import detect
from deep_translator import GoogleTranslator
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import torch
import pandas as pd
import numpy as np

# 📌 Cargar modelo y GPU 
device = "cuda" if torch.cuda.is_available() else "cpu"
model = SentenceTransformer('all-MiniLM-L6-v2', device=device)

# 📌 Cargar dataset con embeddings
embeddings_file = r"C:\Users\solan\Downloads\get_data_from_songs\src\clean_df_embeddings.pkl"
df = pd.read_pickle(embeddings_file)

# 📌 Convertir embeddings a arrays NumPy para mejorar rendimiento
df['embedding'] = df['embedding'].apply(lambda x: np.array(x))

def translate_to_english(text):
    """Detecta el idioma y traduce al inglés si es necesario."""
    detected_lang = detect(text)
    if detected_lang != 'en':  # Si el texto no está en inglés, traducimos
        translated_text = GoogleTranslator(source='auto', target='en').translate(text)
        print(f"🌍 Traducido '{text}' ➝ '{translated_text}'")
        return translated_text
    return text  # Si ya está en inglés, no hacemos nada

def translate_to_spanish(text):
    """Traduce la letra de la canción al español."""
    if pd.notna(text):  # Solo traducimos si hay letra disponible
        return GoogleTranslator(source='en', target='es').translate(text)
    return "Traducción no disponible"

def search_songs(user_query, top_n=5):
    """
    Búsqueda semántica de canciones con traducción automática si el usuario escribe en otro idioma.
    """
    # 📌 Traducir la consulta si es necesario
    translated_query = translate_to_english(user_query)

    # 📌 Convertir la consulta en embedding
    query_embedding = model.encode(translated_query, convert_to_tensor=True).cpu().numpy().reshape(1, -1)

    # 📌 Calcular similitud con todas las canciones
    df['similarity'] = df['embedding'].apply(lambda x: cosine_similarity(x.reshape(1, -1), query_embedding)[0][0])

    # 📌 Ordenar por mayor similitud y devolver las mejores coincidencias
    top_songs = df.sort_values(by="similarity", ascending=False).head(top_n).copy()

    # 📌 Traducir las letras de las canciones al español
    top_songs['translated_lyrics'] = top_songs['processed_lyrics'].apply(lambda x: translate_to_spanish(x[:500]) if isinstance(x, str) else "")

    return top_songs[['artist_name', 'song_name', 'spotify_url', 'processed_lyrics', 'translated_lyrics', 'similarity']]

# 🔍 Prueba con una consulta en español
user_input = "Canciones para bailar con mis amigas"
resultados = search_songs(user_input, top_n=5)

# 📌 Mostrar resultados de forma clara
for index, row in resultados.iterrows():
    print("="*80)
    print(f"🎵 **Canción:** {row['song_name']}")
    print(f"🎤 **Artista:** {row['artist_name']}")
    print(f"🔗 **Spotify:** {row['spotify_url']}")
    print(f"✅ **Similaridad:** {row['similarity']:.4f}")
    
    # Evitar errores con valores NaN
    lyrics = str(row['processed_lyrics']) if pd.notna(row['processed_lyrics']) else "Letra no disponible"
    translated_lyrics = str(row['translated_lyrics']) if pd.notna(row['translated_lyrics']) else "Traducción no disponible"

    print("\n📜 **Letra Original:**\n", lyrics[:800], "...")  # Se imprimen los primeros 800 caracteres
    print("\n🌍 **Traducción Española:**\n", translated_lyrics[:800], "...")
    print("="*80, "\n")


In [ ]:
# 🔍 Prueba con una consulta en español
user_input = "me encanta viajar sola" 
resultados = search_songs(user_input, top_n=5)

# 📌 Mostrar resultados de forma clara
for index, row in resultados.iterrows():
    print("="*80)
    print(f"🎵 **Canción:** {row['song_name']}")
    print(f"🎤 **Artista:** {row['artist_name']}")
    print(f"🔗 **Spotify:** {row['spotify_url']}")
    print(f"✅ **Similaridad:** {row['similarity']:.4f}")
    
    # Evitar errores con valores NaN
    lyrics = str(row['processed_lyrics']) if pd.notna(row['processed_lyrics']) else "Letra no disponible"
    translated_lyrics = str(row['translated_lyrics']) if pd.notna(row['translated_lyrics']) else "Traducción no disponible"

    print("\n📜 **Letra Original:**\n", lyrics[:800], "...")  # Se imprimen los primeros 800 caracteres
    print("\n🌍 **Traducción Española:**\n", translated_lyrics[:800], "...")
    print("="*80, "\n")



In [ ]:
# 🔍 Prueba con una consulta en español
user_input = "sin música la vida sería un error" 
resultados = search_songs(user_input, top_n=5)

# 📌 Mostrar resultados de forma clara
for index, row in resultados.iterrows():
    print("="*80)
    print(f"🎵 **Canción:** {row['song_name']}")
    print(f"🎤 **Artista:** {row['artist_name']}")
    print(f"🔗 **Spotify:** {row['spotify_url']}")
    print(f"✅ **Similaridad:** {row['similarity']:.4f}")
    
    # Evitar errores con valores NaN
    lyrics = str(row['processed_lyrics']) if pd.notna(row['processed_lyrics']) else "Letra no disponible"
    translated_lyrics = str(row['translated_lyrics']) if pd.notna(row['translated_lyrics']) else "Traducción no disponible"

    print("\n📜 **Letra Original:**\n", lyrics[:800], "...")  # Se imprimen los primeros 800 caracteres
    print("\n🌍 **Traducción Española:**\n", translated_lyrics[:800], "...")
    print("="*80, "\n")

In [ ]:
# 🔍 Prueba con una consulta en español
user_input = "He cometido el peor de los pecados que un hombre puede cometer: no he sido feliz" 
resultados = search_songs(user_input, top_n=5)

# 📌 Mostrar resultados de forma clara
for index, row in resultados.iterrows():
    print("="*80)
    print(f"🎵 **Canción:** {row['song_name']}")
    print(f"🎤 **Artista:** {row['artist_name']}")
    print(f"🔗 **Spotify:** {row['spotify_url']}")
    print(f"✅ **Similaridad:** {row['similarity']:.4f}")
    
    # Evitar errores con valores NaN
    lyrics = str(row['processed_lyrics']) if pd.notna(row['processed_lyrics']) else "Letra no disponible"
    translated_lyrics = str(row['translated_lyrics']) if pd.notna(row['translated_lyrics']) else "Traducción no disponible"

    print("\n📜 **Letra Original:**\n", lyrics[:800], "...")  # Se imprimen los primeros 800 caracteres
    print("\n🌍 **Traducción Española:**\n", translated_lyrics[:800], "...")
    print("="*80, "\n")

# all-mpnet-base-v2 - 768 Emb

**Descripción del Modelo:**
El modelo **all-mpnet-base-v2** es conocido por su equilibrio entre precisión y eficiencia. Genera embeddings de alta calidad con una dimensión de 768, lo que lo hace adecuado para una variedad de tareas de procesamiento de lenguaje natural, incluyendo la búsqueda semántica y la comparación de textos.

**Ventajas:**
- **Alta Precisión:** Ofrece una excelente precisión en la comprensión semántica de textos.
- **Eficiencia:** Es más rápido y consume menos recursos que modelos más grandes como **all-roberta-large-v1**.
- **Versatilidad:** Adecuado para tareas de búsqueda semántica y comparación de textos.

**Desventajas:**
- **Menor Capacidad de Comprensión:** Aunque es preciso, no captura el contexto y la semántica tan profundamente como modelos más grandes.
- **Limitaciones en Textos Largos:** Puede no ser tan efectivo en la comprensión de textos muy largos y detallados.

**Razón para No Usarlo en el Proyecto:**
Para este proyecto, que implica la recomendación de canciones basadas en letras y estados de ánimo, se requiere un modelo que pueda capturar de manera precisa y profunda el contexto y la semántica de las letras de las canciones. Aunque **all-mpnet-base-v2** funciona dando unas similitudes más alta, su capacidad de comprensión semántica es menor en comparación con modelos más avanzados como **all-roberta-large-v1** y **sentence-t5-base**. Estos modelos ofrecen una mejor comprensión semántica y precisión, lo que los hace más adecuados para las necesidades del proyecto.

In [ ]:
import pandas as pd
import torch
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from deep_translator import GoogleTranslator
from langdetect import detect

# 📌 Ruta del dataset con los embeddings generados
embeddings_file_768 = r"C:\Users\solan\Downloads\get_data_from_songs\src\clean_df_embeddings_768.pkl"

# 📌 Cargar el dataset con embeddings
df = pd.read_pickle(embeddings_file_768)

# 📌 Convertir embeddings a NumPy arrays para mejorar rendimiento
df['embedding'] = df['embedding'].apply(lambda x: np.array(x))

# 📌 Cargar el modelo (mismo que usamos para generar embeddings)
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"⚡ Usando: {device.upper()}")
model = SentenceTransformer('all-mpnet-base-v2', device=device)

# 📌 Función para traducir la frase del usuario si no está en inglés
def translate_to_english(text):
    detected_lang = detect(text)
    if detected_lang != 'en':  
        translated_text = GoogleTranslator(source='auto', target='en').translate(text)
        print(f"🌍 Traducido '{text}' ➝ '{translated_text}'")
        return translated_text
    return text  

# 📌 Función para traducir las letras de las canciones al español
def translate_to_spanish(text):
    if pd.notna(text):  # Solo traducimos si hay letra disponible
        return GoogleTranslator(source='en', target='es').translate(text)
    return "Traducción no disponible"

# 📌 Función para buscar canciones según una frase de usuario
def search_songs(user_query, top_n=5):
    """
    Encuentra las mejores coincidencias en el dataset según la frase ingresada por el usuario.
    - user_query: Texto del usuario.
    - top_n: Número de canciones recomendadas.
    """
    # 📌 Traducir la frase si no está en inglés
    translated_query = translate_to_english(user_query)

    # 📌 Convertir la frase en embedding
    query_embedding = model.encode(translated_query, convert_to_tensor=True).cpu().numpy()

    # 📌 Calcular similitud con todas las canciones
    df['similarity'] = df['embedding'].apply(lambda x: cosine_similarity([x], [query_embedding])[0][0])

    # 📌 Ordenar por mayor similitud y devolver las mejores coincidencias
    top_songs = df.sort_values(by="similarity", ascending=False).head(top_n).copy()

    # 📌 Traducir las letras de las canciones al español
    top_songs['translated_lyrics'] = top_songs['processed_lyrics'].apply(
        lambda x: translate_to_spanish(x[:500]) if isinstance(x, str) else "Traducción no disponible"
    )

    return top_songs[['artist_name', 'song_name', 'spotify_url', 'processed_lyrics', 'translated_lyrics', 'similarity']]

# 🔍 Prueba con una frase del usuario
user_input = "canciones para bailar con mis amigas"
resultados = search_songs(user_input, top_n=5)

# 📌 Mostrar resultados de forma clara
for index, row in resultados.iterrows():
    print("="*80)
    print(f"🎵 **Canción:** {row['song_name']}")
    print(f"🎤 **Artista:** {row['artist_name']}")
    print(f"🔗 **Spotify:** {row['spotify_url']}")
    print(f"✅ **Similaridad:** {row['similarity']:.4f}")

    # Asegurar que processed_lyrics no sea NaN y convertirlo en string
    lyrics = str(row['processed_lyrics']) if pd.notna(row['processed_lyrics']) else "Letra no disponible"
    translated_lyrics = str(row['translated_lyrics']) if pd.notna(row['translated_lyrics']) else "Traducción no disponible"

    print("\n📜 **Letra Original:**\n", lyrics[:500], "...")
    print("\n🌍 **Traducción Española:**\n", translated_lyrics)
    print("="*100, "\n")


In [ ]:
# 🔍 Prueba con una frase del usuario
user_input = "Me encanta viajar sola"
resultados = search_songs(user_input, top_n=5)

# 📌 Mostrar resultados de forma clara
for index, row in resultados.iterrows():
    print("="*80)
    print(f"🎵 **Canción:** {row['song_name']}")
    print(f"🎤 **Artista:** {row['artist_name']}")
    print(f"🔗 **Spotify:** {row['spotify_url']}")
    print(f"✅ **Similaridad:** {row['similarity']:.4f}")

    # Asegurar que processed_lyrics no sea NaN y convertirlo en string
    lyrics = str(row['processed_lyrics']) if pd.notna(row['processed_lyrics']) else "Letra no disponible"
    translated_lyrics = str(row['translated_lyrics']) if pd.notna(row['translated_lyrics']) else "Traducción no disponible"

    print("\n📜 **Letra Original:**\n", lyrics[:500], "...")
    print("\n🌍 **Traducción Española:**\n", translated_lyrics)
    print("="*100, "\n")

In [ ]:
# 🔍 Prueba con una consulta en español
user_input = "sin música la vida sería un error" 
resultados = search_songs(user_input, top_n=5)

# 📌 Mostrar resultados de forma clara
for index, row in resultados.iterrows():
    print("="*80)
    print(f"🎵 **Canción:** {row['song_name']}")
    print(f"🎤 **Artista:** {row['artist_name']}")
    print(f"🔗 **Spotify:** {row['spotify_url']}")
    print(f"✅ **Similaridad:** {row['similarity']:.4f}")
    
    # Evitar errores con valores NaN
    lyrics = str(row['processed_lyrics']) if pd.notna(row['processed_lyrics']) else "Letra no disponible"
    translated_lyrics = str(row['translated_lyrics']) if pd.notna(row['translated_lyrics']) else "Traducción no disponible"

    print("\n📜 **Letra Original:**\n", lyrics[:800], "...")  # Se imprimen los primeros 800 caracteres
    print("\n🌍 **Traducción Española:**\n", translated_lyrics[:800], "...")
    print("="*100, "\n")

In [ ]:
# 🔍 Prueba con una consulta en español
user_input = "He cometido el peor de los pecados que un hombre puede cometer: no he sido feliz" 
resultados = search_songs(user_input, top_n=5)

# 📌 Mostrar resultados de forma clara
for index, row in resultados.iterrows():
    print("="*80)
    print(f"🎵 **Canción:** {row['song_name']}")
    print(f"🎤 **Artista:** {row['artist_name']}")
    print(f"🔗 **Spotify:** {row['spotify_url']}")
    print(f"✅ **Similaridad:** {row['similarity']:.4f}")
    
    # Evitar errores con valores NaN
    lyrics = str(row['processed_lyrics']) if pd.notna(row['processed_lyrics']) else "Letra no disponible"
    translated_lyrics = str(row['translated_lyrics']) if pd.notna(row['translated_lyrics']) else "Traducción no disponible"

    print("\n📜 **Letra Original:**\n", lyrics[:800], "...")  # Se imprimen los primeros 800 caracteres
    print("\n🌍 **Traducción Española:**\n", translated_lyrics[:800], "...")
    print("="*100, "\n")

## sentence-t5-base

**Descripción del Modelo:**
El modelo **sentence-t5-base** es conocido por su capacidad para comprender y generar texto de manera eficiente. Está optimizado para tareas de búsqueda semántica y generación de texto, lo que lo hace muy versátil en aplicaciones de procesamiento de lenguaje natural.

**Ventajas:**
- **Comprensión Semántica:** Excelente en la comprensión del significado completo de las frases, no solo de palabras individuales.
- **Generación de Texto:** Puede generar texto coherente y relevante, lo que es útil para tareas que requieren respuestas generativas.
- **Equilibrio:** Ofrece un buen balance entre precisión y eficiencia, siendo más rápido que modelos más grandes.

**Desventajas:**
- **No Específico para Música:** Aunque es potente en comprensión semántica, no está específicamente entrenado en datos musicales, lo que puede limitar su rendimiento en tareas muy especializadas.
- **Menor Precisión en Conceptos Abstractos:** Aunque es bueno en comprensión semántica, puede no captar tan bien los conceptos abstractos y matices como otros modelos más avanzados.

**Razón para No Usarlo:**
Aunque **sentence-t5-base** fue nuestra segunda opción debido a su capacidad para comprender el significado completo de las frases, finalmente optamos por **all-roberta-large-v1**. Los resultados obtenidos con **all-roberta-large-v1** fueron significativamente mejores en la captura de conceptos abstractos y matices en las letras de las canciones. La capacidad de **all-roberta-large-v1** para entender el contexto profundo y los detalles de las letras nos convenció de que era la mejor opción para nuestro recomendador de canciones.

In [ ]:
import pandas as pd
import torch
from tqdm import tqdm
from sentence_transformers import SentenceTransformer
import time

# 📌 Ruta del dataset y archivo de embeddings
file_path = r"C:\Users\solan\Downloads\get_data_from_songs\src\clean_df.csv"
embeddings_file_t5 = r"C:\Users\solan\Downloads\get_data_from_songs\src\clean_df_embeddings_t5.pkl"

# 📌 Cargar dataset
df = pd.read_csv(file_path)

# 📌 Crear la columna 'embedding' si no existe
if 'embedding' not in df.columns:
    df['embedding'] = None  # Evita errores al asignar embeddings más tarde

# 📌 Usar GPU si está disponible
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"⚡ Usando: {device.upper()}")  
model = SentenceTransformer('sentence-t5-base', device=device)  # 📌 Modelo con máxima calidad semántica

# 📌 Crear texto combinado para embeddings
df['combined_text'] = (
    df['song_name'].astype(str) + " " +
    df['artist_name'].astype(str) + " " +
    df['processed_lyrics'].astype(str) + " " +
    df['combined_genres'].astype(str) + " " +
    df['playlists_names'].astype(str) + " " +
    df['album_release_date'].astype(str) + " " +
    df['language'].astype(str)
)

# 📌 Parámetros de procesamiento en bloques
batch_size = 10000  
total_rows = len(df)

print(f"🚀 Total de canciones en el dataset: {total_rows}")
print(f"📌 Procesando en bloques de {batch_size} canciones por batch.")

# 📌 Generar embeddings en bloques de 10,000 canciones
for start in range(0, total_rows, batch_size):
    end = min(start + batch_size, total_rows)
    start_time = time.time()  # ⏳ Iniciar contador de tiempo
    print(f"\n⚡ [Batch {start}-{end}] Iniciando procesamiento...")

    # 📌 Obtener lista de textos
    text_batch = df.iloc[start:end]['combined_text'].tolist()

    # 📌 Evitar procesar si el lote está vacío
    if len(text_batch) == 0:
        print(f"❌ [Batch {start}-{end}] No hay datos para procesar. Saltando...")
        continue  

    print(f"⏳ [Batch {start}-{end}] Generando embeddings en GPU...")

    try:
        embeddings_batch = model.encode(
            text_batch,
            batch_size=32,  # 🔥 Máxima calidad sin sobrecargar GPU
            show_progress_bar=True
        )
    except Exception as e:
        print(f"❌ Error en batch {start}-{end}: {str(e)}")
        continue  

    print(f"✅ [Batch {start}-{end}] Embeddings generados. Guardando...")

    # 📌 Convertir cada embedding a lista
    embeddings_batch_list = [emb.tolist() for emb in embeddings_batch]  
    num_rows = end - start  # Número de filas en este batch

    # 📌 Ajustar el tamaño de la lista si hay desajuste con el número de filas
    if len(embeddings_batch_list) > num_rows:
        embeddings_batch_list = embeddings_batch_list[:num_rows]  
    elif len(embeddings_batch_list) < num_rows:
        embeddings_batch_list += [[None] * len(embeddings_batch_list[0])] * (num_rows - len(embeddings_batch_list))

    # 📌 Asignar los embeddings al DataFrame usando iloc y pd.Series
    df.iloc[start:end, df.columns.get_loc('embedding')] = pd.Series(embeddings_batch_list, index=df.index[start:end])

    # 📌 Guardar en archivo para evitar pérdida de progreso
    df.to_pickle(embeddings_file_t5)

    elapsed_time = time.time() - start_time  # ⏱ Tiempo tomado
    print(f"✅ [Batch {start}-{end}] Guardado parcial exitoso en {elapsed_time:.2f} segundos.\n")

    # 📌 Pequeña pausa para evitar sobrecarga de GPU
    time.sleep(2)  

print("🚀 Todos los embeddings generados con el nuevo modelo y guardados correctamente.")


In [ ]:
import pandas as pd
import torch
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from deep_translator import GoogleTranslator
from langdetect import detect

# 📌 Ruta del archivo de embeddings
embeddings_file_t5 = r"C:\Users\solan\Downloads\get_data_from_songs\src\clean_df_embeddings_t5.pkl"

# 📌 Cargar el dataset con los embeddings
df = pd.read_pickle(embeddings_file_t5)

# 📌 Cargar modelo
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"⚡ Usando: {device.upper()}")
model = SentenceTransformer('sentence-t5-base', device=device)

def translate_to_english(text):
    """Traduce la frase del usuario a inglés si no está en inglés."""
    detected_lang = detect(text)
    if detected_lang != 'en':
        translated_text = GoogleTranslator(source='auto', target='en').translate(text)
        print(f"🌍 Traducido '{text}' ➝ '{translated_text}'")
        return translated_text
    return text

def search_songs(user_query, top_n=5):
    """
    Busca canciones en base a una frase de usuario.
    
    - user_query: Texto ingresado por el usuario (ejemplo: "Me gusta conducir en invierno").
    - top_n: Número de canciones recomendadas.
    """
    # 📌 Traducir la consulta si es necesario
    translated_query = translate_to_english(user_query)

    # 📌 Convertir la consulta en embedding
    query_embedding = model.encode(translated_query, convert_to_tensor=True).cpu().numpy()

    # 📌 Calcular similitud con todas las canciones
    df['similarity'] = df['embedding'].apply(lambda x: cosine_similarity([x], [query_embedding])[0][0])

    # 📌 Ordenar por mayor similitud y devolver las mejores coincidencias
    top_songs = df.sort_values(by="similarity", ascending=False).head(top_n)

    return top_songs[['artist_name', 'song_name', 'spotify_url', 'processed_lyrics', 'similarity']]

# 🔍 Prueba con una consulta
user_input = "mis amigos y amigas son los mejores"
resultados = search_songs(user_input, top_n=5)

# 📌 Mostrar resultados
print(resultados)


In [ ]:
import pandas as pd
import torch
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from deep_translator import GoogleTranslator
from langdetect import detect

# 📌 Cargar el dataset con los embeddings
embeddings_file_t5 = r"C:\Users\solan\Downloads\get_data_from_songs\src\clean_df_embeddings_t5.pkl"
df = pd.read_pickle(embeddings_file_t5)

# 📌 Convertir embeddings a array NumPy (MUY IMPORTANTE PARA RENDIMIENTO)
df['embedding'] = df['embedding'].apply(lambda x: np.array(x))
embedding_matrix = np.vstack(df['embedding'].values)  # Convierte todos los embeddings en un array

# 📌 Cargar modelo en GPU si está disponible
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"⚡ Usando: {device.upper()}")
model = SentenceTransformer('sentence-t5-base', device=device)

def translate_to_english(text):
    """Traduce la frase del usuario a inglés si no está en inglés."""
    detected_lang = detect(text)
    if detected_lang != 'en':
        translated_text = GoogleTranslator(source='auto', target='en').translate(text)
        print(f"🌍 Traducido '{text}' ➝ '{translated_text}'")
        return translated_text
    return text

def translate_to_spanish(text):
    """Traduce la letra de la canción al español si está en inglés."""
    if pd.notna(text):  # Solo traducimos si hay letra disponible
        return GoogleTranslator(source='en', target='es').translate(text)
    return "Traducción no disponible"

def search_songs(user_query, top_n=5):
    """Busca canciones en base a una frase del usuario, con cálculos optimizados."""
    
    # 📌 Traducir la consulta si es necesario
    translated_query = translate_to_english(user_query)

    # 📌 Convertir la consulta en embedding
    query_embedding = model.encode(translated_query, convert_to_tensor=True).cpu().numpy().reshape(1, -1)

    # 📌 Calcular similitud en un solo paso (MUCHO MÁS RÁPIDO)
    similarities = cosine_similarity(query_embedding, embedding_matrix)[0]

    # 📌 Agregar resultados a un nuevo dataframe
    df['similarity'] = similarities

    # 📌 Ordenar por mayor similitud y devolver las mejores coincidencias
    top_songs = df.sort_values(by="similarity", ascending=False).head(top_n)

    # 📌 Traducir las letras al español y agregar columna
    top_songs['translated_lyrics'] = top_songs['processed_lyrics'].apply(translate_to_spanish)

    return top_songs[['artist_name', 'song_name', 'spotify_url', 'processed_lyrics', 'translated_lyrics', 'similarity']]

# 🔍 PRUEBA de velocidad con una consulta
user_input = "relojes derretidos en el tiemp"
resultados = search_songs(user_input, top_n=5)

# 📌 Mostrar resultados de forma clara
for index, row in resultados.iterrows():
    print("\n" + "="*100)
    print(f"🎵 **Canción:** {row['song_name']}")
    print(f"🎤 **Artista:** {row['artist_name']}")
    print(f"🔗 **Spotify:** {row['spotify_url']}")
    print(f"✅ **Similaridad:** {row['similarity']:.4f}")

    # Asegurar que processed_lyrics no sea NaN y convertirlo en string
    lyrics = str(row['processed_lyrics']) if pd.notna(row['processed_lyrics']) else "Letra no disponible"
    translated_lyrics = str(row['translated_lyrics']) if pd.notna(row['translated_lyrics']) else "Traducción no disponible"

    print("\n📜 **Letra Original (Primeros 800 caracteres):**\n", lyrics[:800], "...")
    print("\n🌍 **Traducción Española (Primeros 800 caracteres):**\n", translated_lyrics[:800], "...")
    print("="*100)  


In [ ]:
# 🔍 PRUEBA de velocidad con una consulta
user_input = "Me encanta viajar sola"
resultados = search_songs(user_input, top_n=5)

# 📌 Mostrar resultados de forma clara
for index, row in resultados.iterrows():
    print("\n" + "="*100)
    print(f"🎵 **Canción:** {row['song_name']}")
    print(f"🎤 **Artista:** {row['artist_name']}")
    print(f"🔗 **Spotify:** {row['spotify_url']}")
    print(f"✅ **Similaridad:** {row['similarity']:.4f}")

    # Asegurar que processed_lyrics no sea NaN y convertirlo en string
    lyrics = str(row['processed_lyrics']) if pd.notna(row['processed_lyrics']) else "Letra no disponible"
    translated_lyrics = str(row['translated_lyrics']) if pd.notna(row['translated_lyrics']) else "Traducción no disponible"

    print("\n📜 **Letra Original (Primeros 800 caracteres):**\n", lyrics[:800], "...")
    print("\n🌍 **Traducción Española (Primeros 800 caracteres):**\n", translated_lyrics[:800], "...")
    print("="*100)  


In [ ]:
# 🔍 Prueba con una consulta en español
user_input = "sin música la vida sería un error" 
resultados = search_songs(user_input, top_n=5)

# 📌 Mostrar resultados de forma clara
for index, row in resultados.iterrows():
    print("="*80)
    print(f"🎵 **Canción:** {row['song_name']}")
    print(f"🎤 **Artista:** {row['artist_name']}")
    print(f"🔗 **Spotify:** {row['spotify_url']}")
    print(f"✅ **Similaridad:** {row['similarity']:.4f}")
    
    # Evitar errores con valores NaN
    lyrics = str(row['processed_lyrics']) if pd.notna(row['processed_lyrics']) else "Letra no disponible"
    translated_lyrics = str(row['translated_lyrics']) if pd.notna(row['translated_lyrics']) else "Traducción no disponible"

    print("\n📜 **Letra Original:**\n", lyrics[:800], "...")  # Se imprimen los primeros 800 caracteres
    print("\n🌍 **Traducción Española:**\n", translated_lyrics[:800], "...")
    print("="*100, "\n")

In [ ]:

# 🔍 Prueba con una consulta en español
user_input = "He cometido el peor de los pecados que un hombre puede cometer: no he sido feliz" 
resultados = search_songs(user_input, top_n=5)

# 📌 Mostrar resultados de forma clara
for index, row in resultados.iterrows():
    print("="*80)
    print(f"🎵 **Canción:** {row['song_name']}")
    print(f"🎤 **Artista:** {row['artist_name']}")
    print(f"🔗 **Spotify:** {row['spotify_url']}")
    print(f"✅ **Similaridad:** {row['similarity']:.4f}")
    
    # Evitar errores con valores NaN
    lyrics = str(row['processed_lyrics']) if pd.notna(row['processed_lyrics']) else "Letra no disponible"
    translated_lyrics = str(row['translated_lyrics']) if pd.notna(row['translated_lyrics']) else "Traducción no disponible"

    print("\n📜 **Letra Original:**\n", lyrics[:800], "...")  # Se imprimen los primeros 800 caracteres
    print("\n🌍 **Traducción Española:**\n", translated_lyrics[:800], "...")
    print("="*100, "\n")

# Elección del Modelo Final
Después de evaluar varios modelos, decidimos quedarnos con **all-roberta-large-v1** debido a su alta capacidad de comprensión semántica y su rendimiento en textos largos y detallados. Aunque es más pesado y lento que otros modelos, su precisión y capacidad para capturar el contexto profundo de las letras de canciones lo hacen ideal para nuestro recomendador.

# Conclusión
El modelo **all-roberta-large-v1** ha demostrado ser el más adecuado para nuestro sistema de recomendación de canciones basado en letras y estados de ánimo. Su capacidad para entender el contexto y la semántica de las frases completas nos permite ofrecer recomendaciones precisas y relevantes a los usuarios.

# Implementación del Modelo Final
En esta sección, implementamos el modelo **all-roberta-large-v1** para generar embeddings de las letras de canciones y realizar búsquedas semánticas eficientes utilizando FAISS.

In [ ]:
import faiss
import numpy as np
import pandas as pd
import torch
import time
from sentence_transformers import SentenceTransformer

# 📌 Rutas de archivos
file_path = r"C:\Users\solan\Downloads\get_data_from_songs\src\cleaned_songs_data_v2.csv"
faiss_index_path = r"C:\Users\solan\Downloads\get_data_from_songs\src\lyrics_embeddings_faiss_IP.index"

# 📌 Cargar dataset
df = pd.read_csv(file_path)

# 📌 Usar GPU si está disponible
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"⚡ Usando: {device.upper()}")
model = SentenceTransformer('sentence-transformers/all-roberta-large-v1', device=device)  # 🔥 Máxima calidad

# 📌 Crear texto combinado para embeddings (usamos `processed_lyrics`)
df['combined_text'] = (
    df['song_name'].astype(str) + " " +
    df['artist_name'].astype(str) + " " +
    df['processed_lyrics'].astype(str) + " " +
    df['combined_genres'].astype(str) + " " +
    df['playlists_names'].astype(str) + " " +
    df['album_release_date'].astype(str) + " " +
    df['language'].astype(str)
)

# 📌 Parámetros optimizados para evitar sobrecalentamiento
batch_size = 2000  # 🔹 Reducido de 5000 a 2000
pause_time = 30  # ⏸️ Pausa de 30 segundos entre lotes
total_rows = len(df)

print(f"🚀 Total de canciones en el dataset: {total_rows}")
print(f"📌 Procesando en bloques de {batch_size} canciones por batch con pausas de {pause_time}s.")

embeddings_list = []

# 📌 Generar embeddings en bloques con pausas
for start in range(0, total_rows, batch_size):
    end = min(start + batch_size, total_rows)
    start_time = time.time()
    print(f"\n⚡ [Batch {start}-{end}] Iniciando procesamiento...")

    text_batch = df.iloc[start:end]['combined_text'].tolist()

    if len(text_batch) == 0:
        print(f"❌ [Batch {start}-{end}] No hay datos para procesar. Saltando...")
        continue  

    print(f"⏳ [Batch {start}-{end}] Generando embeddings en GPU...")

    try:
        embeddings_batch = model.encode(
            text_batch,
            batch_size=16,  # 🔹 Se mantiene en 16 para no saturar GPU
            show_progress_bar=True
        )
        embeddings_list.extend(embeddings_batch)
    except Exception as e:
        print(f"❌ Error en batch {start}-{end}: {str(e)}")
        continue  

    elapsed_time = time.time() - start_time
    print(f"✅ [Batch {start}-{end}] Embeddings generados en {elapsed_time:.2f} segundos.\n")

    # ⏸️ Pausa para evitar sobrecalentamiento
    print(f"⏳ Esperando {pause_time}s para enfriar la CPU/GPU...\n")
    time.sleep(pause_time)

# 📌 Convertir embeddings a NumPy y crear FAISS index
print("🚀 Guardando embeddings y optimizando FAISS...")

embeddings_np = np.array(embeddings_list, dtype=np.float32)
d = embeddings_np.shape[1]

# 📌 Crear índice FAISS optimizado para búsquedas rápidas
index = faiss.IndexFlatIP(d)
index.add(embeddings_np)

# 📌 Guardar el índice FAISS optimizado
faiss.write_index(index, faiss_index_path)

print(f"✅ FAISS `IndexFlatIP` guardado en {faiss_index_path}, listo para búsquedas semánticas precisas.")


In [ ]:
import pandas as pd
import torch
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from deep_translator import GoogleTranslator
from langdetect import detect

# 📌 Ruta del archivo de embeddings generados con RoBERTa
embeddings_file_roberta = r"C:\Users\solan\Downloads\get_data_from_songs\src\embeddings_roberta2.pkl"

# 📌 Cargar el dataset con los embeddings
df = pd.read_pickle(embeddings_file_roberta)

# 📌 Cargar modelo RoBERTa
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"⚡ Usando: {device.upper()}")
model = SentenceTransformer('all-roberta-large-v1', device=device)

def translate_to_english(text):
    """Traduce la frase del usuario a inglés si no está en inglés."""
    detected_lang = detect(text)
    if detected_lang != 'en':
        translated_text = GoogleTranslator(source='auto', target='en').translate(text)
        print(f"🌍 Traducido '{text}' ➝ '{translated_text}'")
        return translated_text
    return text

def search_songs(user_query, top_n=5):
    """
    Busca canciones en base a una frase de usuario.
    
    - user_query: Texto ingresado por el usuario (ejemplo: "Me gusta conducir en invierno").
    - top_n: Número de canciones recomendadas.
    """
    # 📌 Traducir la consulta si es necesario
    translated_query = translate_to_english(user_query)

    # 📌 Convertir la consulta en embedding
    query_embedding = model.encode(translated_query, convert_to_tensor=True).cpu().numpy()

    # 📌 Asegurar que los embeddings en el DataFrame son listas
    df['embedding'] = df['embedding'].apply(lambda x: x if isinstance(x, list) else list(x))

    # 📌 Calcular similitud con todas las canciones
    df['similarity'] = df['embedding'].apply(lambda x: cosine_similarity([x], [query_embedding])[0][0])

    # 📌 Ordenar por mayor similitud y devolver las mejores coincidencias
    top_songs = df.sort_values(by="similarity", ascending=False).head(top_n)

    return top_songs[['artist_name', 'song_name', 'spotify_url', 'processed_lyrics', 'similarity']]

# 🔍 Prueba con una consulta
user_input = "Comprad el libro de mi hermano, cabrones. Cuando sea famoso y tenga una serie en Netflix diréis que lo habéis leído"
resultados = search_songs(user_input, top_n=5)

# 📌 Mostrar resultados
print(resultados)


* Tarda to much :
Vamos a probar con cosine_similarity() en un solo paso:

In [ ]:
import pandas as pd
import torch
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from deep_translator import GoogleTranslator
from langdetect import detect

# 📌 Ruta del archivo de embeddings generados con RoBERTa
embeddings_file_roberta = r"C:\Users\solan\Downloads\get_data_from_songs\src\embeddings_roberta2.pkl"

# 📌 Cargar el dataset con los embeddings
df = pd.read_pickle(embeddings_file_roberta)

# 📌 Convertir embeddings a NumPy arrays (para mejor rendimiento)
df['embedding'] = df['embedding'].apply(lambda x: np.array(x))

# 📌 Cargar modelo RoBERTa en GPU si está disponible
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"⚡ Usando: {device.upper()}")

model = SentenceTransformer('all-roberta-large-v1', device=device)

def translate_to_english(text):
    """Traduce la frase del usuario a inglés si no está en inglés."""
    detected_lang = detect(text)
    if detected_lang != 'en':
        translated_text = GoogleTranslator(source='auto', target='en').translate(text)
        print(f"🌍 Traducido '{text}' ➝ '{translated_text}'")
        return translated_text
    return text

def translate_to_spanish(text):
    """Traduce una letra de inglés a español."""
    return GoogleTranslator(source='en', target='es').translate(text) if isinstance(text, str) else "Traducción no disponible"

def search_songs(user_query, top_n=5):
    """
    Busca canciones en base a una frase del usuario.
    
    - user_query: Texto ingresado por el usuario (ejemplo: "Me gusta conducir en invierno").
    - top_n: Número de canciones recomendadas.
    """
    # 📌 Traducir la consulta si es necesario
    translated_query = translate_to_english(user_query)

    # 📌 Convertir la consulta en embedding
    query_embedding = model.encode(translated_query, convert_to_tensor=True).cpu().numpy()

    # 📌 Calcular similitud con todas las canciones
    df['similarity'] = df['embedding'].apply(lambda x: cosine_similarity([x], [query_embedding])[0][0])

    # 📌 Ordenar por mayor similitud y devolver las mejores coincidencias
    top_songs = df.sort_values(by="similarity", ascending=False).head(top_n).copy()

    # 📌 Traducir las letras de las canciones al español
    top_songs['translated_lyrics'] = top_songs['processed_lyrics'].apply(lambda x: translate_to_spanish(x[:500]) if isinstance(x, str) else "Traducción no disponible")

    return top_songs[['artist_name', 'song_name', 'spotify_url', 'processed_lyrics', 'translated_lyrics', 'similarity']]

# 🔍 Prueba con una consulta
user_input = "he pedido matrimonio a mi novia durante el rodaje de una película"
resultados = search_songs(user_input, top_n=5)

# 📌 Mostrar resultados de forma clara
for index, row in resultados.iterrows():
    print("="*80)
    print(f"🎵 **Canción:** {row['song_name']}")
    print(f"🎤 **Artista:** {row['artist_name']}")
    print(f"🔗 **Spotify:** {row['spotify_url']}")
    print(f"✅ **Similaridad:** {row['similarity']:.4f}")

    # 🔹 Asegurar que las letras sean cadenas de texto válidas
    original_lyrics = str(row['processed_lyrics']) if isinstance(row['processed_lyrics'], str) else "Letra no disponible"
    translated_lyrics = str(row['translated_lyrics']) if isinstance(row['translated_lyrics'], str) else "Traducción no disponible"

    print("\n📜 **Letra en Inglés:**\n", original_lyrics, "...")
    print("\n🌍 **Traducción Española:**\n", translated_lyrics)
    print("="*500, "\n")


In [ ]:
df.head()

In [ ]:
# 🔍 PRUEBA de velocidad con una consulta
user_input = "Me encanta viajar sola"
resultados = search_songs(user_input, top_n=5)

# 📌 Mostrar resultados de forma clara
for index, row in resultados.iterrows():
    print("\n" + "="*100)
    print(f"🎵 **Canción:** {row['song_name']}")
    print(f"🎤 **Artista:** {row['artist_name']}")
    print(f"🔗 **Spotify:** {row['spotify_url']}")
    print(f"✅ **Similaridad:** {row['similarity']:.4f}")

    # Asegurar que processed_lyrics no sea NaN y convertirlo en string
    lyrics = str(row['processed_lyrics']) if pd.notna(row['processed_lyrics']) else "Letra no disponible"
    translated_lyrics = str(row['translated_lyrics']) if pd.notna(row['translated_lyrics']) else "Traducción no disponible"

    print("\n📜 **Letra Original (Primeros 800 caracteres):**\n", lyrics[:800], "...")
    print("\n🌍 **Traducción Española (Primeros 800 caracteres):**\n", translated_lyrics[:800], "...")
    print("="*100)  


In [ ]:
# 🔍 Prueba con una consulta en español
user_input = "sin música la vida sería un error" 
resultados = search_songs(user_input, top_n=5)

# 📌 Mostrar resultados de forma clara
for index, row in resultados.iterrows():
    print("="*80)
    print(f"🎵 **Canción:** {row['song_name']}")
    print(f"🎤 **Artista:** {row['artist_name']}")
    print(f"🔗 **Spotify:** {row['spotify_url']}")
    print(f"✅ **Similaridad:** {row['similarity']:.4f}")
    
    # Evitar errores con valores NaN
    lyrics = str(row['processed_lyrics']) if pd.notna(row['processed_lyrics']) else "Letra no disponible"
    translated_lyrics = str(row['translated_lyrics']) if pd.notna(row['translated_lyrics']) else "Traducción no disponible"

    print("\n📜 **Letra Original:**\n", lyrics[:800], "...")  # Se imprimen los primeros 800 caracteres
    print("\n🌍 **Traducción Española:**\n", translated_lyrics[:800], "...")
    print("="*100, "\n")

In [ ]:

# 🔍 Prueba con una consulta en español
user_input = "He cometido el peor de los pecados que un hombre puede cometer: no he sido feliz" 
resultados = search_songs(user_input, top_n=5)

# 📌 Mostrar resultados de forma clara
for index, row in resultados.iterrows():
    print("="*80)
    print(f"🎵 **Canción:** {row['song_name']}")
    print(f"🎤 **Artista:** {row['artist_name']}")
    print(f"🔗 **Spotify:** {row['spotify_url']}")
    print(f"✅ **Similaridad:** {row['similarity']:.4f}")
    
    # Evitar errores con valores NaN
    lyrics = str(row['processed_lyrics']) if pd.notna(row['processed_lyrics']) else "Letra no disponible"
    translated_lyrics = str(row['translated_lyrics']) if pd.notna(row['translated_lyrics']) else "Traducción no disponible"

    print("\n📜 **Letra Original:**\n", lyrics[:800], "...")  # Se imprimen los primeros 800 caracteres
    print("\n🌍 **Traducción Española:**\n", translated_lyrics[:800], "...")
    print("="*100, "\n")

In [ ]:
# 🔍 Prueba con una consulta
user_input = "hard metal feliz"
resultados = search_songs(user_input, top_n=5)

# 📌 Mostrar resultados de forma clara
for index, row in resultados.iterrows():
    print("="*80)
    print(f"🎵 **Canción:** {row['song_name']}")
    print(f"🎤 **Artista:** {row['artist_name']}")
    print(f"🔗 **Spotify:** {row['spotify_url']}")
    print(f"✅ **Similaridad:** {row['similarity']:.4f}")

    # 🔹 Asegurar que las letras sean cadenas de texto válidas
    original_lyrics = str(row['processed_lyrics']) if isinstance(row['processed_lyrics'], str) else "Letra no disponible"
    translated_lyrics = str(row['translated_lyrics']) if isinstance(row['translated_lyrics'], str) else "Traducción no disponible"

    print("\n📜 **Letra Original:**\n", original_lyrics[:500], "...")
    print("\n🌍 **Traducción Española:**\n", translated_lyrics)
    print("="*80, "\n")

### NO BertLyrics - Este modelo intentamos usarlo pero estaba disponible y se vio que Roberta estaba reentrenado con él acercándose más a nuestra finalidad


In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch

# Cargar modelo y tokenizer
model_name = "brunokreiner/lyrics-bert"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# Crear embedding para una frase de prueba
sentence = "I feel so lonely tonight"
inputs = tokenizer(sentence, return_tensors="pt")

with torch.no_grad():
    outputs = model(**inputs)

# Obtener los embeddings de la última capa
embeddings = outputs.last_hidden_state[:, 0, :].squeeze().numpy()

print("Embedding shape:", embeddings.shape)


In [ ]:
import torch

# Frase del usuario
user_sentence = "I feel so lonely tonight"

# Tokenización
inputs = tokenizer(user_sentence, return_tensors="pt", padding=True, truncation=True)

# Obtener embeddings
with torch.no_grad():
    outputs = model(**inputs)

# Extraer el embedding del CLS token
user_embedding = outputs.last_hidden_state[:, 0, :].squeeze().numpy()

print("📌 Embedding generado:", user_embedding.shape)


In [ ]:
import pandas as pd
import torch
import numpy as np
from transformers import AutoTokenizer, AutoModel

# 📌 Ruta del dataset
file_path = r"C:\Users\solan\Downloads\get_data_from_songs\src\final_df.csv"

# 📌 Cargar el dataset
df = pd.read_csv(file_path)

# 📌 Seleccionar solo la columna de letras procesadas
df = df[['recording_id', 'processed_lyrics']].dropna()

# 📌 Cargar el modelo de Hugging Face
model_name = "brunokreiner/lyrics-bert"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# 📌 Pasar el modelo a GPU si está disponible
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

# 📌 Función para generar embeddings
def get_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512).to(device)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state[:, 0, :].cpu().numpy().squeeze()

# 📌 Generar los embeddings para todas las canciones
embeddings = []
for lyrics in df['processed_lyrics']:
    embedding = get_embedding(lyrics)
    embeddings.append(embedding)

# 📌 Convertir a numpy array y guardar
embeddings_array = np.array(embeddings)
np.save("lyrics_embeddings.npy", embeddings_array)

# 📌 Guardar un dataframe con IDs y embeddings
df_embeddings = pd.DataFrame({
    'recording_id': df['recording_id'],
    'embedding': list(embeddings)
})
df_embeddings.to_pickle("lyrics_embeddings.pkl")

print("✅ ¡Embeddings generados y guardados correctamente!")


In [ ]:
import pandas as pd
import torch
import numpy as np
from transformers import AutoTokenizer, AutoModel
from deep_translator import GoogleTranslator
from langdetect import detect
from sklearn.metrics.pairwise import cosine_similarity

# 📌 Rutas de los archivos
embeddings_file = "lyrics_embeddings.pkl"  # Archivo con embeddings
metadata_file = "final_df.csv"  # Archivo con datos completos de las canciones
model_name = "brunokreiner/lyrics-bert"  # Modelo

# 📌 Cargar el dataset con embeddings
df_embeddings = pd.read_pickle(embeddings_file)

# 📌 Cargar el dataset completo con metadatos (nombre de canción, artista, etc.)
df_metadata = pd.read_csv(metadata_file)

# 📌 Fusionar embeddings con metadatos usando "recording_id"
df = df_embeddings.merge(df_metadata, on="recording_id", how="left")

# 📌 Cargar modelo y tokenizer
device = "cuda" if torch.cuda.is_available() else "cpu"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name).to(device)

# 📌 Función para traducir la frase del usuario a inglés si es necesario
def translate_to_english(text):
    detected_lang = detect(text)
    if detected_lang != 'en':  
        translated_text = GoogleTranslator(source='auto', target='en').translate(text)
        print(f"🌍 Traducido '{text}' ➝ '{translated_text}'")
        return translated_text
    return text

# 📌 Función para obtener el embedding de la frase del usuario
def get_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512).to(device)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state[:, 0, :].cpu().numpy().squeeze()

# 📌 Función para traducir los primeros caracteres de la letra de la canción al español
def translate_lyrics_snippet(lyrics):
    if pd.notna(lyrics) and isinstance(lyrics, str):
        snippet = lyrics[:300]  # Primeros 300 caracteres
        translated_snippet = GoogleTranslator(source='en', target='es').translate(snippet)
        return translated_snippet
    return "Traducción no disponible"

# 📌 Función para buscar canciones según la frase del usuario
def search_songs(user_query, top_n=5):
    """Encuentra canciones en base a la frase ingresada por el usuario"""
    
    # 🔎 Traducir la frase si es necesario
    translated_query = translate_to_english(user_query)
    
    # 🔎 Convertir la frase en embedding
    query_embedding = get_embedding(translated_query).reshape(1, -1)
    
    # 📌 Calcular similitudes con todas las canciones
    song_embeddings = np.vstack(df['embedding'].values)  # Convertir embeddings a NumPy array
    similarities = cosine_similarity(query_embedding, song_embeddings)[0]
    
    # 📌 Agregar la similitud al dataframe
    df['similarity'] = similarities
    
    # 📌 Ordenar por mayor similitud y seleccionar las mejores coincidencias
    top_songs = df.sort_values(by="similarity", ascending=False).head(top_n)
    
    return top_songs[['recording_id', 'song_name', 'artist_name', 'spotify_url', 'processed_lyrics', 'similarity']]

# 🔍 Prueba con una frase del usuario
user_input = "mis amigos son los mejores"
resultados = search_songs(user_input, top_n=5)

# 📌 Mostrar resultados en tu formato preferido
for index, row in resultados.iterrows():
    print("="*80)
    print(f"🎵 **Canción:** {row['song_name']}")
    print(f"🎤 **Artista:** {row['artist_name']}")
    print(f"🔗 **Spotify URL:** {row['spotify_url'] if pd.notna(row['spotify_url']) else 'Sin URL'}")
    print(f"✅ **Similaridad:** {row['similarity']:.4f}")
    
    # Mostrar letra original (máximo 800 caracteres)
    lyrics = row['processed_lyrics'] if pd.notna(row['processed_lyrics']) else "Letra no disponible"
    print("\n📜 **Letra Original (Primeros 800 caracteres):**")
    print(lyrics[:800] + "..." if len(lyrics) > 800 else lyrics)

    # Traducir los primeros 300 caracteres de la letra
    translated_snippet = translate_lyrics_snippet(lyrics)
    print("\n🌍 **Traducción Española (Primeros 300 caracteres):**")
    print(translated_snippet)
    
    print("="*80 + "\n")


In [ ]:
import pandas as pd
import torch
import numpy as np
from transformers import AutoTokenizer, AutoModel
from sklearn.metrics.pairwise import cosine_similarity

# 📌 Rutas de los archivos
embeddings_file = "lyrics_embeddings.pkl"  # Archivo con los embeddings de las canciones
model_name = "brunokreiner/lyrics-bert"  # Modelo

# 📌 Cargar el dataset con los embeddings
df = pd.read_pickle(embeddings_file)

# 📌 Cargar modelo y tokenizer
device = "cuda" if torch.cuda.is_available() else "cpu"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name).to(device)

# 📌 Función para obtener el embedding de la frase del usuario
def get_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512).to(device)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state[:, 0, :].cpu().numpy().squeeze()

# 📌 Función para buscar canciones según la frase del usuario
def search_songs(user_query, top_n=5):
    """Encuentra canciones en base a la frase ingresada por el usuario"""
    
    # 🔎 Convertir la frase en embedding
    query_embedding = get_embedding(user_query).reshape(1, -1)
    
    # 📌 Calcular similitudes con todas las canciones
    song_embeddings = np.vstack(df['embedding'].values)  # Convertir embeddings a NumPy array
    similarities = cosine_similarity(query_embedding, song_embeddings)[0]
    
    # 📌 Agregar la similitud al dataframe
    df['similarity'] = similarities
    
    # 📌 Ordenar por mayor similitud y seleccionar las mejores coincidencias
    top_songs = df.sort_values(by="similarity", ascending=False).head(top_n)
    
    return top_songs[['recording_id', 'song_name', 'artist_name', 'spotify_url', 'processed_lyrics', 'similarity']]

# 🔍 Prueba con una frase del usuario
user_input = "I feel broken and alone"
resultados = search_songs(user_input, top_n=5)

# 📌 Mostrar resultados en tu formato preferido
for index, row in resultados.iterrows():
    print("="*80)
    print(f"🎵 **Canción:** {row['song_name']}")
    print(f"🎤 **Artista:** {row['artist_name']}")
    print(f"🔗 **Spotify URL:** {row['spotify_url'] if pd.notna(row['spotify_url']) else 'Sin URL'}")
    print(f"✅ **Similaridad:** {row['similarity']:.4f}")
    
    # Mostrar letra (máximo 800 caracteres)
    lyrics = row['processed_lyrics'] if pd.notna(row['processed_lyrics']) else "Letra no disponible"
    print("\n📜 **Letra Original (Primeros 800 caracteres):**")
    print(lyrics[:800] + "..." if len(lyrics) > 800 else lyrics)
    print("="*80 + "\n")


Decarto este

RED NEURONAL PARA JUNTAR EMBEDDINGS DE T5 - ROBERTA Y MPNET

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

# 📌 Cargar datasets con embeddings de diferentes modelos
df_t5 = pd.read_pickle("clean_df_embeddings_t5.pkl")
df_roberta = pd.read_pickle("clean_df_embeddings_roberta.pkl")
df_mpnet = pd.read_pickle("clean_df_embeddings_768.pkl")  # MPNet

# 📌 Unir los datasets usando recording_id como clave
df = df_t5.merge(df_roberta, on="recording_id", suffixes=("_t5", "_roberta"))
df = df.merge(df_mpnet, on="recording_id")

# 📌 Convertir embeddings en arrays NumPy y normalizar
def normalize_embedding(embedding_list):
    embedding_array = np.array(embedding_list)
    return embedding_array / np.linalg.norm(embedding_array)

df["embedding_t5"] = df["embedding_t5"].apply(normalize_embedding)
df["embedding_roberta"] = df["embedding_roberta"].apply(normalize_embedding)
df["embedding_mpnet"] = df["embedding"].apply(normalize_embedding)  # MPNet

# 📌 Concatenar los embeddings en un solo vector
df["combined_embedding"] = df.apply(lambda row: np.concatenate([
    row["embedding_t5"],
    row["embedding_roberta"],
    row["embedding_mpnet"]
]), axis=1)

# 📌 Guardar el dataset combinado
df[["recording_id", "combined_embedding"]].to_pickle("clean_df_embeddings_combined.pkl")

print("✅ Embeddings fusionados y guardados correctamente")


In [ ]:
import faiss
import numpy as np
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModel

# 📌 Rutas de tus archivos
csv_file = r"C:\Users\solan\Downloads\get_data_from_songs\src\clean_df.csv"
embeddings_file = r"C:\Users\solan\Downloads\get_data_from_songs\src\clean_df_embeddings_combined.pkl"

# 📌 Cargar dataset
df = pd.read_csv(csv_file)

# 📌 Cargar embeddings
df_embeddings = pd.read_pickle(embeddings_file)

# 📌 Unir ambos datasets por 'recording_id'
df = df.merge(df_embeddings, on="recording_id", how="inner")

# 📌 Convertir embeddings a un array NumPy (FAISS necesita float32)
embedding_matrix = np.vstack(df['combined_embedding'].values).astype("float32")

# 📌 Crear índice FAISS
index = faiss.IndexFlatL2(embedding_matrix.shape[1])  # Índice de similitud L2
index.add(embedding_matrix)

# 📌 Cargar modelo para obtener embeddings de consulta
model_name = "sentence-transformers/all-mpnet-base-v2"
device = "cuda" if torch.cuda.is_available() else "cpu"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name).to(device)


def get_combined_embedding(text):
    """Genera el embedding combinado usando los mismos modelos que se usaron en FAISS"""
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512).to(device)

    with torch.no_grad():
        mpnet_output = mpnet_model(**inputs).last_hidden_state[:, 0, :].cpu().numpy().squeeze()
        t5_output = t5_model(**inputs).last_hidden_state[:, 0, :].cpu().numpy().squeeze()
        roberta_output = roberta_model(**inputs).last_hidden_state[:, 0, :].cpu().numpy().squeeze()

    # 📌 Concatenar los embeddings en un solo vector de 2560 dimensiones (768+768+1024)
    combined_embedding = np.concatenate([mpnet_output, t5_output, roberta_output])

    return combined_embedding


# 📌 Función para obtener embedding de la frase del usuario
def get_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512).to(device)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state[:, 0, :].cpu().numpy().astype("float32")

def search_songs_faiss(user_query, top_n=5):
    query_embedding = get_combined_embedding(user_query).reshape(1, -1)
    distances, indices = index.search(query_embedding, top_n)  # FAISS realiza la búsqueda

    # 📌 Extraer resultados del dataframe
    results = df.iloc[indices[0]].copy()
    return results


# 🔍 Prueba con una frase del usuario
user_input = "I feel broken and alone"
resultados = search_songs_faiss(user_input, top_n=5)

# 📌 Mostrar resultados en tu formato preferido
for index, row in resultados.iterrows():
    print("="*80)
    print(f"🎵 **Canción:** {row['song_name']}")
    print(f"🎤 **Artista:** {row['artist_name']}")
    print(f"🔗 **Spotify URL:** {row['spotify_url'] if pd.notna(row['spotify_url']) else 'Sin URL'}")
    print(f"✅ **Similaridad:** {row['similarity']:.4f}")
    
    # Mostrar letra (máximo 800 caracteres)
    lyrics = row['processed_lyrics'] if pd.notna(row['processed_lyrics']) else "Letra no disponible"
    print("\n📜 **Letra Original (Primeros 800 caracteres):**")
    print(lyrics[:800] + "..." if len(lyrics) > 800 else lyrics)
    print("="*80 + "\n")


In [ ]:
# 📌 Verificar dimensión del índice FAISS
print(f"FAISS Index Dim: {index.d}")

# 📌 Generar el embedding de la consulta y verificar su dimensión
query_embedding = get_embedding("I feel broken and alone").reshape(1, -1)
print(f"Query Embedding Dim: {query_embedding.shape[1]}")


In [ ]:
print(df_embeddings.columns)


Tarda demasiado - pruebo faiss

In [ ]:
# Descripción estadística de las columnas numéricas
df.describe()